# Zakupki website scraping for Piotr

The aim of this notebook is to scrape details of each contract hosted on the Russian Zakupki public sector contract awarding website.

The input for this project will be the Zakupki URL. This code can be run on different dates to pull fresh contract data.

Method:
1.   Identify the number of pages of contracts to be scraped (using the contract filters provided).
2.   Iterate through each page, scraping the registration number of each contract.
3.   Access the website for each contract by placing the registraion number in the URL.
4.   Scrape the details for each contract and add them to a list of Contracts dataclasses.
5.   Format these Contract objects as a dataframe and output the dataframe to a csv file.


The output of this project will be the CSV file, with each row representing a new contract from the webstie.


### Section 1: Setup

In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from datetime import date
from dataclasses import dataclass
from tqdm import tqdm
from dateutil import parser
from threading import Thread
import pandas as pd
from datetime import datetime, date, timedelta
import logging
import http.client
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor, as_completed
import math
from os import walk
import json
import numpy as np
import csv


In [2]:
## Finding the memory leak

from collections import Counter
import linecache
import os
import tracemalloc

def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))


tracemalloc.start()

In [3]:
logging = False

if logging:

    http.client.HTTPConnection.debuglevel = 1

    # You must initialize logging, otherwise you'll not see debug output.
    logging.basicConfig()
    logging.getLogger().setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

### Section 2: Determine Number of pages to scrape
Test connection to the website and determine number of pages to scrape

In [4]:
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# @lru_cache(maxsize=None)
def getPage(tempURL):
  # If User-Agent is not set to custom, the website will know a Python script is accessing it and block some of the request

  response = session.get(tempURL, headers={'User-Agent': 'Custom'})
  return BeautifulSoup(response.content, "html.parser")

In [5]:
# Getting the dates we want to scrape.

url="https://zakupki.gov.ru/epz/contractfz223/search/results.html?morphology=on&sortBy=BY_UPDATE_DATE&pageNumber={}&sortDirection=false&recordsPerPage=_500&showLotsInfoHidden=false&statuses_0=on&statuses_1=on&statuses=0%2C1&priceFrom=1000000&currencyId=-1&contract223DateFrom={}&contract223DateTo={}&customerPlace=5277383"

#ORIGINAL
# https://zakupki.gov.ru/epz/contract/search/results.html?morphology=on&search-filter=%D0%94%D0%B0%D1%82%D0%B5+%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D1%8F&fz44=on&contractStageList_0=on&contractStageList_1=on&contractStageList=0%2C1&contractPriceFrom=1000000&contractCurrencyID=-1&budgetLevelsIdNameHidden=%7B%7D&customerPlace=5277383&customerPlaceCodes=66000000000&contractDateFrom={}&contractDateTo={}&selectedLaws=FZ44&sortBy=UPDATE_DATE&pageNumber={}&sortDirection=false&recordsPerPage=_500&showLotsInfoHidden=false"

startDate = date(2016, 1, 1)
endDate = date(2016, 1, 31)
days = timedelta(days=1)

startDateBefore = startDate

calendar=[]

while startDate<=endDate:
  calendar.append(startDate.strftime('%d.%m.%Y'))
  startDate+=days

print("Created {} dates".format(len(calendar)))
startDate = date(2016, 1, 1)
endDate = date(2016, 1, 31)
days = timedelta(days=1)

startDateBefore = startDate

calendar=[]

while startDate<=endDate:
  calendar.append(startDate.strftime('%d.%m.%Y'))
  startDate+=days

print("Created {} dates".format(len(calendar)))

Created 31 dates
Created 1827 dates



### Section 3: Scrape each registration number

Scrape the reg numbers of each contract, so they can be accessed individually

In [6]:
class Page:

    def __init__(self, day, pageNum, pagefile):

        self.day = day
        self.pageNum = pageNum
        self.pagefile = pagefile


In [7]:
def getContracts(page):

    regNumbersList = []

    # Obtain a list of all the sections of HTML containing a contract in the web page
    listOfContracts = page.find_all("div", {"class": "registry-entry__header-mid__number"})
    
    # Segment the registration number from the URL of each contract
    for contract in listOfContracts:
        href = contract.find("a")['href']
        regNum = href[href.index("Number=")+7:]
        regNumbersList.append(regNum)

    return regNumbersList

In [8]:
def progress(idx, data):

    x_ = int(((idx+1) * 100) / len(data))
    y_ = idx % math.ceil(len(data) / 10)
    
    print(" ----\n{}% completed\n----".format(x_)) if y_ == 0 else None

In [9]:
# Getting the web page for all the contracts for each date in the range we want to scrape.

regNumbersDict = {}

def getRegNumbersForDate(i, day):

  if day in regNumbersDict:
    return

  tempURL = url.format(day, day, 1)

  print(tempURL)

  page = getPage(tempURL)

  # Scrape the max number of pages
  try:
    maxPageNum = int(page.select('a[data-pagenumber]')[-2].find("span").text)
    print("{} pages for this day".format(maxPageNum))
  except:
    maxPageNum = 1


  # Leave my variable names alone :(
  totalRegNumbersForThisDay = 0

  for i in range(1, maxPageNum+1):

    # Creating a temporary URL for each page containing contracts
    tempPageURL = url.format(day, day, i)

    # Request the page and format it as a BeautifulSoup object so that we can perform scrapings
    page = getPage(tempPageURL)

    regNumbersList = getContracts(page)

    totalRegNumbersForThisDay += len(regNumbersList)

    regNumbersDict[day] = regNumbersList


  print("Fetched day {} had {} contracts \n".format(day, totalRegNumbersForThisDay), end='')

  del page
  del regNumbersList

  # progress(i, calendar)


In [10]:

## This now has regNumbers caching too 

cachedRegNums = {}

# load the data from the json file
with open('cachedRegNums.json', 'r') as f:
  cachedRegNums = json.load(f)


with ThreadPoolExecutor(max_workers=50) as ex:
  for i, day in enumerate(calendar):
    if day in cachedRegNums:
      regNumbersDict[day] = cachedRegNums[day]
      print("Cached day {} had {} contracts \n".format(day, len(regNumbersDict[day])), end='')
    else:
      ex.submit(getRegNumbersForDate, i, day)

    
combinedRegNumbersDict = {**cachedRegNums, **regNumbersDict}

# print(combinedRegNumbersDict)


with open('cachedRegNums.json', 'w') as f:
  json.dump(combinedRegNumbersDict, f)


tempRegNumbers = list(regNumbersDict.values())

regNumbers = []

for t in tempRegNumbers:
  regNumbers.extend(t)

# print(regNumbers)

print("------------------- \n {} contracts found in total".format(len(regNumbers)))

Cached day 01.01.2016 had 80 contracts 
Cached day 02.01.2016 had 4 contracts 
Cached day 03.01.2016 had 2 contracts 
Cached day 04.01.2016 had 4 contracts 
Cached day 05.01.2016 had 1 contracts 
Cached day 06.01.2016 had 5 contracts 
Cached day 07.01.2016 had 0 contracts 
Cached day 08.01.2016 had 3 contracts 
Cached day 09.01.2016 had 1 contracts 
Cached day 10.01.2016 had 2 contracts 
Cached day 11.01.2016 had 115 contracts 
Cached day 12.01.2016 had 41 contracts 
Cached day 13.01.2016 had 23 contracts 
Cached day 14.01.2016 had 25 contracts 
Cached day 15.01.2016 had 25 contracts 
Cached day 16.01.2016 had 1 contracts 
Cached day 17.01.2016 had 0 contracts 
Cached day 18.01.2016 had 40 contracts 
Cached day 19.01.2016 had 34 contracts 
Cached day 20.01.2016 had 19 contracts 
Cached day 21.01.2016 had 21 contracts 
Cached day 22.01.2016 had 20 contracts 
Cached day 23.01.2016 had 3 contracts 
Cached day 24.01.2016 had 1 contracts 
Cached day 25.01.2016 had 48 contracts 
Cached day 2

The below is just for testing labels etc.

### Section 4: Details scraping

The Contract Dataclass will store the information during scraping.
If any information can't be scraped, default values have been provided in their place

In [12]:
@dataclass
class Contract:

  # TODO: Add reg number to class

  # Main Section
  id: float = 0
  price: float = 0.0
  signed: date = None
  deadline: date = None

  # Tab 1
  method: str = "none"
  procurer: str = "none"
  supplier: str = "none"
  proinn: str = "none"
  supinn: str = "none"
  registered: date = None
  address: str = "none"
  number: str = "none"
  mail: str = "none"

  # Tab 2
  code: float = 0.0
  product: str = "none"
  

  def __repr__(self):
    return "\nContract id= {} \n First tab: price={}, signed={}, deadline={}, method={}, procurer={}, supplier={}, proinn={}, supinn={}, registered={}, address={}, number={}, mail={} \n  Second tab: code={}, product={})".format(self.id, self.price, self.signed, self.deadline, self.method, self.procurer, self.supplier, self.proinn, self.supinn, self.registered, self.address, self.number, self.mail, self.code,  self.product)

  

Method for scraping the data from each contract

In [13]:
def getSectionDict(page):

    sections=page.findAll("section",{"class":"blockInfo__section section"})

    # print([key.findAll("span") for key in sections])

    # Turning the sections into a dictionary that will be easier to work with.
    pairs = [key.findAll("span") for key in sections]

    pairs = list(filter(None, pairs))


    titles = []
    values = []

    for x in pairs:
        if len(x) > 1:
            try:
                titles.append(x[0])
                values.append(x[1])
            except:
                pass

    sectionDict = {titles[i].text.strip() : values[i].text.strip() for i in range(len(titles))}

    return sectionDict


def getTableDict(page, secondTab=False):


    if secondTab:
        sectionOfInterest = page.findAll("div", {"class": "col"})[-1]
    else:
        sectionOfInterest = page

    table = sectionOfInterest.findAll("tr",{"class":"tableBlock__row"})

    # print(table[3])

    headers = [i.text.strip() for i in table[0].findAll("th", {"class":"tableBlock__col tableBlock__col_header"})]
    data = [list(filter(None, [j.strip() for j in i.text.split("\n")])) for i in table[1].findAll("td")]

    if len(headers) == 0:
        headers = [i.text.strip() for i in table[2].findAll("th", {"class":"tableBlock__col tableBlock__col_header"})]
        data = [list(filter(None, [j.strip() for j in i.text.split("\n")])) for i in table[3].findAll("td")]

    if len(headers) == 0:
        headers = [i.text.strip() for i in table[3].findAll("th", {"class":"tableBlock__col tableBlock__col_header"})]
        data = [list(filter(None, [j.strip() for j in i.text.split("\n")])) for i in table[4].findAll("td")]



    # This is hacky.
    if len(data) < len(headers):
        data = [[[]] for i in range(len(headers))]

        

    tableDict = {headers[i] : data[i] for i in range(len(headers))}

    return tableDict

In [14]:
def scrapeData(reg):

  try:
    # Input: reg = one registration number.

    # Different URL from the one above, this accesses more information from Zakupki.
    dir = "https://zakupki.gov.ru/epz/contract/contractCard/{}.html?reestrNumber={}"

    # Getting the web page for the given contract
    tempDir = dir.format("common-info", reg)
    page = getPage(tempDir)

    # We probably don't need this with the method I've used below.
    contractTypeTwo = False

    # Enter the text here that should be present to signify the second type of contract.
    if page.findAll(text="Основание заключения контракта с единственным поставщиком"):
      contractTypeTwo = True
      
    id = reg  
    sectionDict = getSectionDict(page)
    firstTableDict = getTableDict(page)

    # print(sectionDict)
    # print(firstTableDict)

    # print(sectionDict, firstTableDict)
    # try:
    #   price=sectionDict["Цена контракта"].replace("\xa0","").replace(",",".").replace("₽","").strip().split()[0]
    # except:
    #   price=sectionDict["Ориентировочное значение цены контракта"].replace("\xa0","").replace(",",".").replace("₽","").strip().split()[0]
    #   try:
    #     price=sectionDict["Максимальное значение цены контракта"].replace("\xa0","").replace(",",".").replace("₽","").strip().split()[0]
    #   except:
    price=page.find('span', {'class':'cardMainInfo__content cost'}).text.strip().replace("₽","").replace(" ","")
      
    signed=sectionDict["Дата заключения контракта"].split()[0]
    deadline=sectionDict["Дата окончания исполнения контракта"].split()[0]
    
    ### fixed issue with method ### 
    try:
      method = sectionDict["Способ определения поставщика (подрядчика, исполнителя)"]
    except:
      if page.findAll(text="Основание заключения контракта с единственным поставщиком"):
            method="Закупка у единственного поставщика (подрядчика, исполнителя)"
            
    procurer=sectionDict["Полное наименование заказчика"]
    supplier=firstTableDict["Организация"][0]

    proinn=sectionDict["ИНН"]

    ### fixed issues for missing values sometimes in the table ###
    
    registered=firstTableDict["Организация"][-1]
    
    ### fixed, testing ###   
    try: 
      if firstTableDict["Организация"][-4]=="КПП:":
          supinn=firstTableDict["Организация"][-5]
      else:
          supinn=firstTableDict["Организация"][-3]
    except:
      supinn=""

    ### fixed issues in lower table ### 
    
    try:
      address=firstTableDict["Адрес места нахождения"]
    except:
      address=firstTableDict["Адрес в стране регистрации"]
    number=firstTableDict["Телефон, электронная почта"][0]
    mail=firstTableDict["Телефон, электронная почта"][1]

    ### details about winner - ALSO, THERE'S OPTION TO SCRAPE SUBCONTRACTORS ### 

    page.decompose()

    ### Second tab ###
    tempDir = dir.format("payment-info-and-target-of-order", reg)
    page = getPage(tempDir)

    ### code stands for the product code, which can be later identified to return industry type ### 
    
    secondTableDict = getTableDict(page, True)
    
    try:
      code = secondTableDict["Позиции по КТРУ, ОКПД2"][1][1:-1]
    except:
      code = ''
        
    product = secondTableDict["Наименование объекта закупки и его характеристики"][0]
    
    # Create the Contract dataclass object and append it to a list of objects.
    # This method means that missing data can be accounted for.
    # print(method)

    contract = Contract(id=id, price=price, signed=signed, deadline=deadline, method=method, procurer=procurer, supplier=supplier, proinn=proinn, supinn=supinn, registered=registered, address=address, number=number, mail=mail, code=code, product=product)
    
    # contracts.append(contract)
    # print('Completed {}'.format(id))

    page.decompose()

    return contract
  except Exception as e:
    failedRegNumbers.append(reg)
    print("Failed to scrape {}".format(reg))
    print(e)

### Section 5: Starting execution
Scrape the contracts themselves using threading

In [15]:
def scrape(reg):
    
    try:
        _ = int(reg)
        # print("Scraping {}".format(reg))
        return [scrapeData(reg)]
    except TypeError:
        
        # TODO make 500 contracts change here.
        
        contracts = []

        

        for idx, r in enumerate(reg):
            # print("Scraping {}".format(r))
            contracts.append(scrapeData(r))
            # progress(i + idx, regNumbers)

        return contracts

In [16]:
progressNum = 0

failedRegNumbers = []

# regNumbers = [3666404326716000002, 3666404326716000002, 1722300731616000034, 1722300731616000034, 2666001041516000469]

print("Starting scrape with {} reg numbers\n".format(len(regNumbers)))

# scrape(regNumbers[:10])

# for contract in contracts:
#     print(contract)

# for regNumber in tqdm(regNumbers[:50]):
#   thread = Thread(target = scrapeData, args = (regNumber,))
#   thread.start()

# regNumbers = ['3662502457421000001']
# regNumbers = ['1665800691921000016']

threading = True

if threading:

    interval = 1

    with ThreadPoolExecutor(max_workers=20) as ex:
        threads = []

        cachedContracts = {}

        with open('cachedContracts.csv', encoding="utf-8") as f:
            cachedContracts = list(csv.reader(f))
        
        cachedContracts = list(filter(None, cachedContracts))
        
        cachedContractRegNums = [row[0] for row in cachedContracts]

        uncachedRegNumbers = list(set(regNumbers) - set(cachedContractRegNums))

        print("{} of {} contracts are uncached. Fetching...".format(len(uncachedRegNumbers), len(regNumbers)))
        
        for i in range(0, len(uncachedRegNumbers), interval):
            tempNumbers = uncachedRegNumbers[i:i+interval]
            # print(tempNumbers)
            threads.append(ex.submit(scrape, tempNumbers))
        

        completed = 0

        for result in tqdm(as_completed(threads)):

            try:
                contracts = result.result()

                completed += interval

                # progress(completed, uncachedRegNumbers)

                formattedContracts = [list(contract.__dict__.values()) for contract in contracts]

                # newCachedContracts = {**cachedContracts, **formattedContracts}

                with open('cachedContracts.csv', 'a', encoding="utf-8", newline='') as f:
                    writer = csv.writer(f)
                    writer.writerows(formattedContracts)
            except Exception as e:
                print(e)
                
            # print("{} finished".format(interval))

else:

    scrape(regNumbers)

# print("Scraped {} contracts".format(len(contracts)))
print("Failed to scrape {} contracts".format(len(failedRegNumbers)))
print(failedRegNumbers) if len(failedRegNumbers) > 0 else None

snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

Starting scrape with 41177 reg numbers

93 of 41177 contracts are uncached. Fetching...


1it [00:03,  3.41s/it]

Failed to scrape 1660900206920000211
'Организация'
'NoneType' object has no attribute '__dict__'
Failed to scrape 2661800178420000132
'Организация'
'NoneType' object has no attribute '__dict__'


3it [00:06,  1.77s/it]

Failed to scrape 2661900214920000260
'Организация'
'NoneType' object has no attribute '__dict__'
Failed to scrape 1665808957720000246
list index out of range
Failed to scrape 2665808158520000020
'Организация'
'NoneType' object has no attribute '__dict__'


...

'NoneType' object has no attribute '__dict__'
Failed to scrape 3660800191517000008
'Наименование объекта закупки и его характеристики'
'NoneType' object has no attribute '__dict__'
Failed to scrape 2661900214920000263
'Организация'
'NoneType' object has no attribute '__dict__'


88it [01:12,  2.64it/s]

Failed to scrape 3666100295320000131
'Наименование объекта закупки и его характеристики'
'NoneType' object has no attribute '__dict__'


90it [01:13,  2.25it/s]

Failed to scrape 1666100918720000043
'Организация'
Failed to scrape 2667143939720000061
'Наименование объекта закупки и его характеристики'
'NoneType' object has no attribute '__dict__'
'NoneType' object has no attribute '__dict__'
Failed to scrape 3662503996520000010
'Наименование объекта закупки и его характеристики'
'NoneType' object has no attribute '__dict__'


### Now that the data is saved to the hard disk, we can run the below code without needing to rerun the scraping process

In [ ]:
cachedContracts = []

with open('cachedContracts.csv', encoding="utf-8") as f:
    cachedContracts = list(csv.reader(f))


df = pd.DataFrame(columns=Contract().__dict__.keys(), data=cachedContracts)

df['signed'] = pd.to_datetime(df['signed'])
df['deadline'] = pd.to_datetime(df['deadline'])

start = datetime.fromordinal(startDateBefore.toordinal()).strftime("%Y-%m-%d")
end = datetime.fromordinal(endDate.toordinal()).strftime("%Y-%m-%d")

print(start, end)

mask = (df['signed'] >= start) & (df['signed'] <= end)

selectedDatesDF = df.loc[mask]

selectedDatesDF.head()

2016-01-01 2020-12-31


,id,price,signed,deadline,method,procurer,supplier,proinn,supinn,registered,address,number,mail,code,product
0,3660800365518000015,"1 974 925,54",2018-07-30,2018-12-31,Электронный аукцион,ЕКАТЕРИНБУРГСКОЕ МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДП...,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СПЕЦ...",6608003655,6664079721,01.09.2011,"['620010, ОБЛ СВЕРДЛОВСКАЯ 66, Г ЕКАТЕРИНБУРГ,...",7-343-2260122,tender@specregion.ru,14.12.30.190,Белье
1,1770989550919007394,"1 460 823,00",2019-05-12,2020-06-30,Закупка у единственного поставщика (подрядчика...,МЕЖРЕГИОНАЛЬНЫЙ ФИЛИАЛ ФЕДЕРАЛЬНОГО КАЗЕННОГО ...,ПЫТАЛЕВ АНДРЕЙ СТЕПАНОВИЧИндивидуальный предпр...,7709895509,773671146984,20.07.2018,['Г МОСКВА 77'],8-912-4150862,pzs_arenda@mail.ru,68.20.12.000,Передача во временное владение и пользование ч...
2,3666005552917000109,"5 347 130,00",2017-09-13,2017-12-31,Закупка у единственного поставщика (подрядчика...,"МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ""ЕКАТЕРИНБУ...","Общество с ограниченной ответственностью ""МО ""...",6660055529,6658126476,08.02.2001,"['620102, г. Екатеринбург, ул. Ясная, 46']",(343) 231-80-10,omt@omt-ural.ru,26.60.13.190,"Устройство терморегулирующее ""Аллон 2001"""
3,3668200734219000001,"3 361 200,00",2019-08-04,2019-09-30,Электронный аукцион,"МУНИЦИПАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""УПРАВЛЕНИЕ ...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЕКАТ...",6682007342,6672249991,01.01.2012,"['620026, ОБЛ СВЕРДЛОВСКАЯ 66, Г ЕКАТЕРИНБУРГ,...",7-343-3787200,102@eta-info.ru,86.90.19.140,Предоставление услуг по организации отдыха и о...
4,3661100106218000013,"1 200 000,00",2018-03-29,2018-12-31,Закупка у единственного поставщика (подрядчика...,"МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ ""КОМБИНАТ ...","ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЭНЕРГОСБЫТ ПЛЮС...",6611001062,5612042824,22.05.2014,"['143421, ОБЛ МОСКОВСКАЯ 50, Р-Н КРАСНОГОРСКИЙ...",8-34355-52497,sesb@ies-holding.com,35.11.10.115,"Электроэнергия, произведенная атомными электро..."


In [ ]:
selectedDatesDF.shape

(41098, 15)

### Section 6: Output

Convert the list of contract classes to a dataframe so that they can be exported to a csv file

In [ ]:
selectedDatesDF.to_csv("zakupki{}to{}.csv".format(startDateBefore, endDate))